In [1]:
%load_ext autoreload
%autoreload 2

from common import *
from config import *
import utils
from models.model import*

module://ipykernel.pylab.backend_inline
@common.py:  
	set random seed
		SEED=35202
	set cuda environment
		torch.__version__              = 1.0.0
		torch.version.cuda             = 9.0.176
		torch.backends.cudnn.version() = 7401
		torch.cuda.device_count()      = 1



In [2]:
config = DefaultConfigs()


# 1. set random seed
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpus
try:
    print('\t\tos[\'CUDA_VISIBLE_DEVICES\']     =', os.environ['CUDA_VISIBLE_DEVICES'])
    NUM_CUDA_DEVICES = len(os.environ['CUDA_VISIBLE_DEVICES'].split(','))
except Exception:
    print('\t\tos[\'CUDA_VISIBLE_DEVICES\']     =', 'None')
    NUM_CUDA_DEVICES = 1
warnings.filterwarnings('ignore')

if not os.path.exists('../results'):
    os.mkdir('../results')

if not os.path.exists(config.logs):
    os.mkdir(config.logs)

log = utils.Logger()
log.open('{0}{1}_log_train.txt'.format(config.logs, config.model_name),mode="a")

log.write("\n-------------------- [START %s] %s\n\n" % (datetime.now().strftime('%Y-%m-%d %H:%M:%S'), '-' * 51))
log.write('                          |------ Train ------|------ Valid ------|----Best Results---|------------|\n')
log.write('mode    iter   epoch    lr|  loss    f1_macro |  loss    f1_macro |  loss    f1_macro | time       |\n')
log.write('----------------------------------------------------------------------------------------------------\n')

		os['CUDA_VISIBLE_DEVICES']     = 0

-------------------- [START 2019-01-18 18:00:34] ---------------------------------------------------

                          |------ Train ------|------ Valid ------|----Best Results---|------------|
mode    iter   epoch    lr|  loss    f1_macro |  loss    f1_macro |  loss    f1_macro | time       |
----------------------------------------------------------------------------------------------------


In [3]:
 # 4.1 mkdirs
if not os.path.exists(config.submit):
    os.makedirs(config.submit)
if not os.path.exists(config.weights + config.model_name + os.sep + 'fold_'+str(config.fold)):
    os.makedirs(config.weights + config.model_name + os.sep + 'fold_'+ str(config.fold))
if not os.path.exists(config.best_models):
    os.mkdir(config.best_models)
if not os.path.exists(config.logs):
    os.mkdir(config.logs)

tqdm.pandas()

start_time = time.time()
train_X, test_X, train_y, word_index = utils.load_and_prec(config)
# embedding_matrix_1 = load_glove(word_index)
# embedding_matrix_2 = load_para(word_index)

total_time = (time.time() - start_time) / 60
print("Took {:.2f} minutes".format(total_time))

  4%|▎         | 46992/1306122 [00:00<00:02, 469918.21it/s]

Train shape :  (1306122, 3)
Test shape :  (56370, 2)


100%|██████████| 56370/56370 [00:00<00:00, 70710.01it/s]


Tokenizing.......
Tokenizing Done!
Took 2.80 minutes


In [4]:

# -------------------------------------------------------
# training
# -------------------------------------------------------
train_preds = np.zeros((len(train_X)))
test_preds = np.zeros((len(test_X)))

x_test_cuda = torch.tensor(test_X, dtype=torch.long).cuda()
test_dataset = torch.utils.data.TensorDataset(x_test_cuda)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False)

splits = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED).split(train_X, train_y))

sigmoid = nn.Sigmoid()
loss_fn = torch.nn.BCEWithLogitsLoss(reduction="sum")

# k-fold
for fold, (train_idx, valid_idx) in enumerate(splits):
    print(f'Fold {fold + 1}')

    # tflogger
    tflogger = utils.TFLogger(os.path.join('results', 'TFlogs',
                                     config.model_name + "_fold{0}_{1}".format(config.fold, fold)))
    # initialize the early_stopping object
    early_stopping = utils.EarlyStopping(patience=7, verbose=True)

    x_train_fold = torch.tensor(train_X[train_idx], dtype=torch.long).cuda()
    y_train_fold = torch.tensor(train_y[train_idx, np.newaxis], dtype=torch.float32).cuda()
    x_val_fold = torch.tensor(train_X[valid_idx], dtype=torch.long).cuda()
    y_val_fold = torch.tensor(train_y[valid_idx, np.newaxis], dtype=torch.float32).cuda()

    model = Baseline_Bidir_LSTM_GRU(config, word_index)
    model.cuda()

    optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)

    train_dataset = torch.utils.data.TensorDataset(x_train_fold, y_train_fold)
    valid_dataset = torch.utils.data.TensorDataset(x_val_fold, y_val_fold)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=config.batch_size, shuffle=False)

    valid_loss = np.inf
    start_time = time.time()
    for epoch in range(config.epochs):
        # train
        lr = utils.get_learning_rate(optimizer)
        train_loss = train(train_loader=train_loader,model=model,loss_fn=loss_fn, optimize=optimizer,
                           epoch=epoch,valid_loss=valid_loss,start=start_time)

        # validate
        valid_loss = evaluate(val_loader=valid_loader, model=model, loss_fn=loss_fn, epoch=epoch,
                            train_loss=train_loss, start_time=start_time)
        test_preds_fold = np.zeros(len(test_X))

        # save model
        utils.save_checkpoint({
            "epoch": epoch,
            "model_name": config.model_name,
            "state_dict": model.state_dict(),
            "optimizer": optimizer.state_dict(),
            "fold": config.fold,
            "kfold": config.fold,
        },config.fold, fold, config)
        # print logs
        print('\r', end='', flush=True)

        log.write("\n")
        time.sleep(0.01)

        # ================================================================== #
        #                        Tensorboard Logging                         #
        # ================================================================== #

        # 1. Log scalar values (scalar summary)
        info = {'Train_loss': train_loss,
                'Valid_loss': valid_loss,
                'Learnging_rate': lr}

        for tag, value in info.items():
            tflogger.scalar_summary(tag, value, epoch)

        # 2. Log values and gradients of the parameters (histogram summary)
        for tag, value in model.named_parameters():
            tag = tag.replace('.', '/')
            tflogger.histo_summary(tag, value.data.cpu().numpy(), epoch)
            tflogger.histo_summary(tag + '/grad', value.grad.data.cpu().numpy(), epoch)
        # -------------------------------------
        # end tflogger

        # ================================================================== #
        #                        Early stopping                         #
        # ================================================================== #
        # early_stopping needs the validation loss to check if it has decresed,
        # and if it has, it will make a checkpoint of the current model
        early_stopping(valid_loss, model)

        if early_stopping.early_stop:
            print("Early stopping")
            break

    # end looping all epochs

    # test
    test_preds_fold = test(test_loader=test_loader, model=model)
    test_preds += test_preds_fold / len(splits)
    # end k-fold

Fold 1
Start loading embedding....................
Embedding matrix shape: (120000, 300)


AttributeError: module 'utils' has no attribute 'get_learning_rate'

In [25]:
m = nn.Sigmoid()
m(y_pred).cpu().numpy()[:,0].shape

(1536,)

In [18]:
torch.nn.Sigmoid([[0]])

TypeError: __init__() takes 1 positional argument but 2 were given